In [1]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

In [2]:
def sigmoidp(x): 
    return 1 / (1 + np.exp(-x))

# Swish aktivasyon fonksiyonu
def Swish(x): 
    return x * sigmoidp(x)

# Swish'in türevi (derivatif)
def dSwish(x): 
    return Swish(x) + sigmoidp(x) * (1 - Swish(x))

In [3]:
# Özellik çıkarma fonksiyonu: Resimden anlamlı istatistiksel özellikler çıkartılır
def extract_features(image_path):
    image = cv2.imread(image_path)  # Resmi oku
    image = cv2.resize(image, (128, 128))  # 128x128 boyutuna getirme
    image = image.astype(np.float32) / 255.0  # Normalizasyon (0-1 arası)
    
    R, G, B = cv2.split(image)  # RGB kanallarını ayırma
    
    # Her kanal için istatistiksel özellikleri hesapla
    def compute_features(channel):
        mean = np.mean(channel)
        entropy = -np.sum(channel * np.log2(channel + 1e-10))  # Entropi hesaplama
        std = np.std(channel)
        skewness = np.mean((channel - mean) ** 3) / (std ** 3 + 1e-10)
        moment = np.var(channel)
        fft_mean = np.mean(np.abs(np.fft.fft2(channel)))  # Fourier dönüşümü ortalaması
        percentile_50 = np.percentile(channel, 50)  # 50. persentil (medyan)
        median = np.median(channel)
        return [mean, entropy, std, skewness, moment, fft_mean, percentile_50, median]
    
    # RGB her kanal için özellikleri çıkar
    features = compute_features(R) + compute_features(G) + compute_features(B)
    
    # Renk indeksleri (Brightness, Saturation, Hue, vb.)
    brightness = (np.mean(R**2 + G**2 + B**2)) / 3
    saturation = np.mean((R - B) / (R + B + 1e-10))
    hue = np.mean((2 * R - G - B) / (G - B + 1e-10))
    coloration = np.mean((R - G) / (R + G + 1e-10))
    redness = np.mean(R**2 / (B * G**3 + 1e-10))
    
    features += [brightness, saturation, hue, coloration, redness]  # Renk indekslerini özelliklere ekle
    
    return np.array(features)

In [4]:
# Eğitim verisi klasörü
train_dir = "C:/Users/User/Desktop/CaltechTinySplit/train"

# Tüm resimler için özellikleri çıkartma
features_list = []

# Klasörler (flamingo ve pizza) içinde gezinme
for class_name in ['flamingo', 'pizza']:
    class_dir = os.path.join(train_dir, class_name)
    
    for filename in os.listdir(class_dir):
        image_path = os.path.join(class_dir, filename)
        
        # Özellikleri çıkartma
        features = extract_features(image_path)
        
        # Bias (1) ekleme
        features_with_bias = np.append(features, 1)
        features_list.append(features_with_bias)

# Özellikleri NumPy array'e dönüştürme
features_array = np.array(features_list)

# Normalizasyon işlemi: Bias dışındaki özellikler üzerinde normalizasyon
features_array[:, :29] = features_array[:, :29] / np.max(features_array[:, :29], axis=0)

# İlk 2 örneği yazdırma (özellikleri kontrol etme)
print("İlk 2 resmin özellikleri (Bias ile):")
for i in range(2):
    print(f"Resim {i+1} Özellikler (Bias eklenmiş): {features_array[i]}")

İlk 2 resmin özellikleri (Bias ile):
Resim 1 Özellikler (Bias eklenmiş): [ 1.00000000e+00  9.40029940e-02  3.52542145e-01 -1.56793699e+00
  1.24285952e-01  4.32301806e-01  1.00000000e+00  1.00000000e+00
  1.00000000e+00  1.04119492e-01  3.94805541e-01 -1.39924210e+00
  1.55871431e-01  3.92371935e-01  1.00000000e+00  1.00000000e+00
  1.00000000e+00  7.73034432e-02  2.97950223e-01 -1.64550229e+00
  8.87743387e-02  3.88544130e-01  1.00000000e+00  1.00000000e+00
  1.00000000e+00 -4.17326197e-02 -7.07938868e-03  7.06733733e-02
  3.56264718e-05  1.00000000e+00]
Resim 2 Özellikler (Bias eklenmiş): [ 0.17539012  0.65730993  0.44702655  1.          0.19983275  0.33682162
  0.09411765  0.09411765  0.23161734  0.74520216  0.45372092  0.654961
  0.20586269  0.28919692  0.13725491  0.13725491  0.29646052  0.64827652
  0.77943217  0.33587968  0.60751445  0.38967597  0.11764706  0.11764706
  0.10492237 -0.72703172 -0.2359236  -1.17333097  0.00816407  1.        ]


In [5]:
# Eğitim verisi klasörü
train_dir = "C:/Users/User/Desktop/CaltechTinySplit/train"

# Flamingo ve pizza klasörlerinin yolları
flamingo_dir = os.path.join(train_dir, 'flamingo')
pizza_dir = os.path.join(train_dir, 'pizza')

# Flamingo resimlerine 0, Pizza resimlerine 1 verecek targets array'ini oluşturma
targets = []

# Flamingo resimleri için 0 ekleyelim
for filename in os.listdir(flamingo_dir):
    targets.append(0)

# Pizza resimleri için 1 ekleyelim
for filename in os.listdir(pizza_dir):
    targets.append(1)

# Targets array'ini NumPy array'e dönüştürme
targets = np.array(targets)

print(targets)
print(len(targets))
weights = np.random.rand(30)*0.01
print(weights)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
95
[0.00905531 0.00787314 0.00454727 0.00851433 0.0039884  0.00866045
 0.00062891 0.00876417 0.0040447  0.00119238 0.00475311 0.00616852
 0.00731731 0.00148721 0.007974   0.00758788 0.00257907 0.00686614
 0.00242615 0.00710212 0.00803882 0.00252278 0.00922562 0.00680808
 0.00225678 0.00247989 0.00901394 0.00122346 0.00264753 0.00977283]


In [6]:
inputs=features_array
x = inputs[i]  # x1, x2 ve bias dahil
print(x)
sum_t = np.dot(x, weights)
print(sum_t)

[ 0.17539012  0.65730993  0.44702655  1.          0.19983275  0.33682162
  0.09411765  0.09411765  0.23161734  0.74520216  0.45372092  0.654961
  0.20586269  0.28919692  0.13725491  0.13725491  0.29646052  0.64827652
  0.77943217  0.33587968  0.60751445  0.38967597  0.11764706  0.11764706
  0.10492237 -0.72703172 -0.2359236  -1.17333097  0.00816407  1.        ]
0.05591444441176718


In [7]:
print(inputs.shape)  # features_array boyutunu kontrol et
print(targets.shape)  # targets boyutunu kontrol et

(95, 30)
(95,)


In [8]:
learning_rate = 0.0001  # Öğrenme oranı
epochs = 1000  # Eğitim için epoch sayısı
inputs = features_array  # Girdi özellikleri

# Eğitim verisini karıştırma
inputs, targets = shuffle(inputs, targets, random_state=0)

# Perceptron algoritması ile eğitim
for epoch in range(epochs):
    inputs, targets = shuffle(inputs, targets)  # Veriyi her epoch başında karıştır
    
    # Her örnek için perceptron öğrenme kuralını uygulama
    for i in range(inputs.shape[0]):
        x = inputs[i]  # Girdi vektörü
        target = targets[i]  # Hedef çıktı
        
        # Swish aktivasyon fonksiyonu ile çıktıyı hesapla
        y = np.dot(x, weights)
        error = target - y  # Lineer kombinasyon (w*x)
        Sw = dSwish(y)  # Swish'in türevini al
        weights += learning_rate * error * Sw * x  # Ağırlık güncelleme
    
    # İsteğe bağlı olarak her epoch'tan sonra hata ya da ağırlıkları yazdır
    if (epoch + 1) % 1000 == 0:  # Her 1000 epoch'ta bir yazdır
        print(f"Epoch {epoch + 1}/{epochs}, Weights: {weights}")

Epoch 1000/1000, Weights: [-0.01241926  0.00778736  0.03588066  0.04330931  0.08486182  0.08343265
 -0.01683208 -0.00869682  0.00524718 -0.04145504  0.02650831  0.03315684
  0.06474684  0.07947641  0.01574674  0.01536062  0.06759059 -0.04297931
 -0.0776451  -0.11775426 -0.06551559  0.05354768  0.12593837  0.12352083
  0.01639217 -0.35472362  0.05796647 -0.04556282  0.00145413 -0.05323397]


In [9]:
# Eğitim verisi klasörü
train_dir = "C:/Users/User/Desktop/CaltechTinySplit/test"


# Tüm resimler için özellikleri çıkartma
features_list = []

# Klasörler (flamingo ve pizza) içinde gezinme
for class_name in ['flamingo', 'pizza']:
    class_dir = os.path.join(train_dir, class_name)
    
    for filename in os.listdir(class_dir):
        image_path = os.path.join(class_dir, filename)
        
        # Özellikleri çıkartma
        features = extract_features(image_path)
        
        # Bias (1) ekleme
        features_with_bias = np.append(features, 1)
        features_list.append(features_with_bias)

# Özellikleri NumPy array'e dönüştürme
features_array = np.array(features_list)

# Normalizasyon işlemi: Bias dışındaki özellikler üzerinde
features_array[:, :29] = features_array[:, :29] / np.max(features_array[:, :29], axis=0)

# İlk 2 örneği yazdırma
print("İlk 2 resmin özellikleri (Bias ile):")
for i in range(2):
    print(f"Resim {i+1} Özellikler (Bias eklenmiş): {features_array[i]}")


İlk 2 resmin özellikleri (Bias ile):
Resim 1 Özellikler (Bias eklenmiş): [ 1.00000000e+00  6.97794800e-01  5.45928347e-01 -1.31933216e+00
  2.98037739e-01  4.83837741e-01  1.00000000e+00  1.00000000e+00
  8.99880261e-01  7.09470693e-01  6.67483839e-01 -1.48747452e+00
  4.45534680e-01  5.02585477e-01  9.84615386e-01  9.84615386e-01
  8.37021170e-01  7.14088571e-01  5.08264114e-01 -2.09147768e+00
  2.58332436e-01  4.69827187e-01  8.38427958e-01  8.38427958e-01
  8.46141285e-01 -6.22105616e-01  3.74777031e-02  3.76557173e-01
  2.03969464e-04  1.00000000e+00]
Resim 2 Özellikler (Bias eklenmiş): [ 5.34258350e-01  9.42386492e-01  5.90138590e-01  7.89372149e-01
  3.48263560e-01  4.94839545e-01  4.16666673e-01  4.16666673e-01
  5.68800833e-01  9.93964756e-01  6.31672002e-01  7.95294948e-01
  3.99009520e-01  4.95568576e-01  5.02564102e-01  5.02564102e-01
  5.75214556e-01  9.14865851e-01  7.82528250e-01  6.62460046e-01
  6.12350492e-01  5.20140227e-01  4.75982534e-01  4.75982534e-01
  3.88631125

In [10]:
train_dir = "C:/Users/User/Desktop/CaltechTinySplit/test"

# Flamingo ve pizza klasörlerinin yolları
flamingo_dir = os.path.join(train_dir, 'flamingo')
pizza_dir = os.path.join(train_dir, 'pizza')

# Flamingo resimlerine 0, Pizza resimlerine 1 verecek targets array'ini oluşturma
targets = []

# Flamingo resimleri için 0 ekleyelim
for filename in os.listdir(flamingo_dir):
    targets.append(0)

# Pizza resimleri için 1 ekleyelim
for filename in os.listdir(pizza_dir):
    targets.append(1)

# Targets array'ini NumPy array'e dönüştürme
targets = np.array(targets)

print(targets)
print(len(targets))
print(weights)

[0 0 0 0 0 0 0 0 1 1 1 1 1 1]
14
[-0.01241926  0.00778736  0.03588066  0.04330931  0.08486182  0.08343265
 -0.01683208 -0.00869682  0.00524718 -0.04145504  0.02650831  0.03315684
  0.06474684  0.07947641  0.01574674  0.01536062  0.06759059 -0.04297931
 -0.0776451  -0.11775426 -0.06551559  0.05354768  0.12593837  0.12352083
  0.01639217 -0.35472362  0.05796647 -0.04556282  0.00145413 -0.05323397]


In [11]:
predictions = []
inputs = features_array
print("\nTesting:")
for i in range(len(inputs)): 
    x = inputs[i]  # Girdi vektörü          
    y = np.dot(x, weights)  # Ağırlıklar ile lineer kombinasyonu hesapla
    prediction =  Swish(y)  # Swish fonksiyonuyla çıktı al
    predicted_class = 1 if prediction > 0.5 else 0  # 0.5'ten büyükse 1, küçükse 0 tahmin et
    predictions.append(predicted_class)  # Tahminleri sakla
    
    # Şekil kontrolü ve çıktıyı yazdırma
    print(f" Target: {targets[i]}, Prediction: {predicted_class}, Raw Output: {prediction:.4f}")

# Doğruluk hesaplama
accuracy = accuracy_score(targets, predictions)
print(f"\nAccuracy: {accuracy * 100:.2f}%")


Testing:
 Target: 0, Prediction: 0, Raw Output: 0.4312
 Target: 0, Prediction: 1, Raw Output: 0.6281
 Target: 0, Prediction: 1, Raw Output: 0.9804
 Target: 0, Prediction: 0, Raw Output: 0.0927
 Target: 0, Prediction: 0, Raw Output: 0.2744
 Target: 0, Prediction: 1, Raw Output: 2.0863
 Target: 0, Prediction: 0, Raw Output: -0.1292
 Target: 0, Prediction: 0, Raw Output: 0.2781
 Target: 1, Prediction: 1, Raw Output: 2.0585
 Target: 1, Prediction: 1, Raw Output: 1.1817
 Target: 1, Prediction: 1, Raw Output: 1.0135
 Target: 1, Prediction: 1, Raw Output: 2.0568
 Target: 1, Prediction: 1, Raw Output: 1.0122
 Target: 1, Prediction: 1, Raw Output: 0.5426

Accuracy: 78.57%
